#Word2Vec IMDB

In [ ]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 10.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.0
    Uninstalling scipy-1.16.0:
      Successfully uninstalled scipy-1.16.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
opencv-python-h

In [ ]:
#Import important libraries
import pandas as pd
import os
import numpy as np

from gensim.models import Word2Vec


Load dataset

In [ ]:
df=pd.read_csv('/content/IMDB Dataset.csv')

In [ ]:
df.head()


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


#Simple data Preprocessing

In [ ]:
df.shape

(50000, 2)

In [ ]:
df['review'][0]


"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [ ]:
df=df.iloc[:10000]
df.head()
#integrated Location iloc

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
df.shape


(10000, 2)

In [ ]:
df['sentiment'].value_counts()

,count
sentiment,
positive,5030
negative,4970


In [ ]:
df.duplicated().sum()

0

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.isnull().sum()

,0
review,0
sentiment,0


#Basic Preprecessing




1. Remove tags-HTML
2. Lower case
3. Remove stopwords




In [ ]:
import re
def remove_tags(raw_text):
    cleaned_text=re.sub(re.compile('<.*?>'),'',raw_text)
    return cleaned_text

In [ ]:
df['review']=df['review'].apply(remove_tags)

In [ ]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming tec...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
df['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.I would say the main appeal of the show is due to the fact that it goes where other shows wo

#stopwords Remove


In [ ]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

sw_list = stopwords.words('english')

# Step 1: lowercase + split
df['review'] = df['review'].apply(lambda x: x.lower().split())

# Step 2: remove stopwords
df['review'] = df['review'].apply(lambda x: [item for item in x if item not in sw_list])



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df['review'][0]

['one',
 'reviewers',
 'mentioned',
 'watching',
 '1',
 'oz',
 'episode',
 'hooked.',
 'right,',
 'exactly',
 'happened',
 'me.the',
 'first',
 'thing',
 'struck',
 'oz',
 'brutality',
 'unflinching',
 'scenes',
 'violence,',
 'set',
 'right',
 'word',
 'go.',
 'trust',
 'me,',
 'show',
 'faint',
 'hearted',
 'timid.',
 'show',
 'pulls',
 'punches',
 'regards',
 'drugs,',
 'sex',
 'violence.',
 'hardcore,',
 'classic',
 'use',
 'word.it',
 'called',
 'oz',
 'nickname',
 'given',
 'oswald',
 'maximum',
 'security',
 'state',
 'penitentary.',
 'focuses',
 'mainly',
 'emerald',
 'city,',
 'experimental',
 'section',
 'prison',
 'cells',
 'glass',
 'fronts',
 'face',
 'inwards,',
 'privacy',
 'high',
 'agenda.',
 'em',
 'city',
 'home',
 'many..aryans,',
 'muslims,',
 'gangstas,',
 'latinos,',
 'christians,',
 'italians,',
 'irish',
 'more....so',
 'scuffles,',
 'death',
 'stares,',
 'dodgy',
 'dealings',
 'shady',
 'agreements',
 'never',
 'far',
 'away.i',
 'would',
 'say',
 'main',
 'ap

In [ ]:
# Select all rows and only the first column (by index), keeping it as a DataFrame
x = df.iloc[:, 0:1]
x

,review
0,"[one, reviewers, mentioned, watching, 1, oz, e..."
1,"[wonderful, little, production., filming, tech..."
2,"[thought, wonderful, way, spend, time, hot, su..."
3,"[basically, there's, family, little, boy, (jak..."
4,"[petter, mattei's, ""love, time, money"", visual..."
...,...
10012,"[pleasantly, surprised, find, lose, friends, a..."
10013,"[moving, film, takes, new, twist, somewhere, b..."
10014,"[apart, beautiful, imagery, thanks, new, zeala..."
10015,"[perfect, series, family, viewing., gather, ar..."


#Encode Sentiment

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
y=encoder.fit_transform(df['sentiment'])

In [ ]:
y

array([1, 1, 1, ..., 0, 0, 1])

# Train Word2Vec

In [ ]:
w2v_model=Word2Vec(sentences=df['review'],vector_size=100,window=5, min_count=2,workers=4)
w2v_model.train(df['review'],total_examples=len(df['review']),epochs=10)

(11026469, 12255850)

#Convert Reviews to Vectors

In [ ]:
def get_review_vector(review,model):
    vectors=[model.wv[word]for word in review if word in model.wv]
    if len(vectors)>0:
       return np.mean(vectors,axis=0)
    else:
        return np.eros(model.vector_size)
df['review_vector'] =df['review'].apply(lambda x: get_review_vector(x,w2v_model))

#Prepare Data for Classification

In [ ]:
df['review_vector']

,review_vector
0,"[-0.19089322, 0.37577587, -0.046396777, 0.1876..."
1,"[-0.3507348, 0.32278535, 0.15829125, 0.5619573..."
2,"[-0.10370758, 0.42944804, -0.15086925, 0.27030..."
3,"[-0.10013792, 0.3774779, 0.13063599, 0.4390617..."
4,"[-0.35868013, 0.42751253, -0.016255572, 0.0246..."
...,...
10012,"[0.032722697, 0.23494163, -0.003186273, 0.4680..."
10013,"[-0.35116273, 0.34147182, -0.20470552, 0.04220..."
10014,"[-0.18727615, 0.40499786, 0.0033841159, 0.3980..."
10015,"[-0.38082746, 0.6191201, -0.07907419, 0.042233..."


In [ ]:
from sklearn.model_selection import train_test_split
X=np.array(df['review_vector'].tolist())
y=df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#RandomForest


In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.7706559839759639


#Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X_train, y_train)


LogisticRegression()

In [ ]:
from sklearn.metrics import accuracy_score
print("Accuracy:", accuracy_score(y_test, clf.predict(X_test)))


Accuracy: 0.8215
